<a href="https://colab.research.google.com/github/dellingam/Coursera_Capstone/blob/master/The_Battle_of_Neighbourhood_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Capstone Project</h1>


Importing the necessary libraries.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

/bin/bash: conda: command not found
Libraries imported.


The Bay Area Companies List was uploaded on the IBM Cloud and is being imported from the same.

In [2]:
data = pd.read_csv('/content/Bay-Area-Companies-List.csv')

In [3]:
data.head()

,Company Name,Tags,Location,Investors,Description,Website,Founded Year,Address,Lat,Long,Company Size,Tech stack,Marketing Stack,Design Stack,Product Stack
0,3scale,"B2B Software and Services,Developer Tool",San Francisco,NaN,Unlock the power of your APIs (owned by RedHat),https://www.3scale.net/,2007.0,"450 Townsend St #204, San Francisco, CA 94107",37.774634,-122.398642,1-12,NaN,NaN,NaN,NaN
1,8tracks,"Music,Consumer Goods and Services",San Francisco,NaN,8tracks is the best place to make and discover...,https://8tracks.com/,2008.0,"51 Sharon St, San Francisco, CA 94114",37.765227,-122.429756,13-60,NaN,NaN,NaN,NaN
2,10 by 10,"B2B Software and Services,Talent Acquisition",San Francisco,Y Combinator,We help independent recruiters and agencies ea...,https://www.10by10.io/,2015.0,"San Francisco, CA",37.774929,-122.419415,1-12,NaN,NaN,NaN,NaN
3,15Five,Employee Engagement,San Francisco,NaN,15Five software elevates the performance and e...,15five.com,2011.0,"12 Gallagher Ln, San Francisco, CA 94103",37.781714,-122.403236,61-150,NaN,NaN,NaN,NaN
4,21Tech,"Real Estate and Construction,Financial Technol...",East Bay,NaN,Enterprise Asset Management: 21Tech has broad ...,21tech.com,1996.0,"1330 Broadway #1530, Oakland, CA 94612",37.803900,-122.270794,13-60,NaN,NaN,NaN,NaN


Feature Selection

In [4]:
# Extracting the necessary columns 
df = data[['Company Name','Location','Lat','Long']]
df.head()

,Company Name,Location,Lat,Long
0,3scale,San Francisco,37.774634,-122.398642
1,8tracks,San Francisco,37.765227,-122.429756
2,10 by 10,San Francisco,37.774929,-122.419415
3,15Five,San Francisco,37.781714,-122.403236
4,21Tech,East Bay,37.803900,-122.270794


In [5]:
# Renaming the columns
df.rename(columns={'Company Name':'Company','Location':'Neighbourhood','Lat':'Latitude','Long':'Longitude'}, inplace=True)
df.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Company,Neighbourhood,Latitude,Longitude
0,3scale,San Francisco,37.774634,-122.398642
1,8tracks,San Francisco,37.765227,-122.429756
2,10 by 10,San Francisco,37.774929,-122.419415
3,15Five,San Francisco,37.781714,-122.403236
4,21Tech,East Bay,37.803900,-122.270794


In [6]:
df.shape

(756, 4)

In [7]:
# Since there are a lot of rows, only the first 200 rows are extracted
df = df.loc[0:200,:]
df.shape

(201, 4)

Plotting all the company offcies in the Bay Ares on the map.

In [8]:
# create map of Bay Area using latitude and longitude values
map_sv = folium.Map(location=[37.7749, -122.4194], zoom_start=12)

# add markers to map
for lat, lng, company, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Company'], df['Neighbourhood']):
    label = '{}, {}'.format(company, neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sv)  
    
map_sv

Using the Foursquare API to explore the regions near the company offices.

In [9]:
CLIENT_ID = 'WXRC2QLPQEFKHGRSJEEUSTDESG4Q4QT3T5GUM5OLDY4GHDOV' # your Foursquare ID
CLIENT_SECRET = 'FNZMZIK4QWDZE5OWGLLFVPCSCOIBP0JZEZETQ44IW1XNYKPP' # your Foursquare Secret
ACCESS_TOKEN = '1YCBHAP5BT025OSK1OFLG5FBHNBMYWJ0ZRCE2QTZNNMELZ24' # your access token
VERSION = '20200801' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WXRC2QLPQEFKHGRSJEEUSTDESG4Q4QT3T5GUM5OLDY4GHDOV
CLIENT_SECRET:FNZMZIK4QWDZE5OWGLLFVPCSCOIBP0JZEZETQ44IW1XNYKPP


In [10]:
radius = 1000
LIMIT = 200


venues = []
for lat, long, neighborhood,company in zip(df['Latitude'], df['Longitude'], df['Neighbourhood'], df['Company']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&oauth_token={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        ACCESS_TOKEN,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()
    results = results["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            company,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))


In [11]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Company', 'Latitude','Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(19151, 8)


,Neighborhood,Company,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,San Francisco,3scale,37.774634,-122.398642,SF Flower Mart,37.775435,-122.400913,Flower Shop
1,San Francisco,3scale,37.774634,-122.398642,Bay Club SF Tennis,37.776571,-122.397948,Tennis Court
2,San Francisco,3scale,37.774634,-122.398642,Local Brewing Co.,37.776550,-122.397171,Brewery
3,San Francisco,3scale,37.774634,-122.398642,Mission Creek Park,37.771538,-122.398753,Park
4,San Francisco,3scale,37.774634,-122.398642,MNTSTUDIO,37.773537,-122.402653,Gym / Fitness Center


In [12]:
# Extracting the rows with Venue Category as a Restaurant
df0 = venues_df.loc[venues_df['VenueCategory'].str.contains('Restaurant') == True]

In [13]:
df0.head()

,Neighborhood,Company,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
5,San Francisco,3scale,37.774634,-122.398642,Dinosaurs,37.775013,-122.403349,Vietnamese Restaurant
9,San Francisco,3scale,37.774634,-122.398642,Cockscomb,37.778578,-122.397395,New American Restaurant
13,San Francisco,3scale,37.774634,-122.398642,Marlowe,37.778289,-122.396685,New American Restaurant
26,San Francisco,3scale,37.774634,-122.398642,Bellota,37.771759,-122.404886,Spanish Restaurant
29,San Francisco,3scale,37.774634,-122.398642,Dumpling Time,37.769812,-122.402460,Dumpling Restaurant


In [14]:
# Extracting the rows with Neighborhood in Bay Area
df1 = df0.loc[venues_df['Neighborhood'].str.contains('Bay') == True]

In [15]:
df1.head()

,Neighborhood,Company,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
402,East Bay,21Tech,37.8039,-122.270794,Golden Lotus Vegetarian Restaurant,37.803290,-122.270473,Vegetarian / Vegan Restaurant
405,East Bay,21Tech,37.8039,-122.270794,Nature Vegetarian Restaurant,37.802157,-122.270983,Vegetarian / Vegan Restaurant
406,East Bay,21Tech,37.8039,-122.270794,Abura-Ya,37.805959,-122.267693,Japanese Restaurant
410,East Bay,21Tech,37.8039,-122.270794,Tay Ho Restaurant & Bar,37.802062,-122.269573,Vietnamese Restaurant
413,East Bay,21Tech,37.8039,-122.270794,Shan Dong,37.800500,-122.270046,Chinese Restaurant


In [16]:
# Sorting the rows by Venue Category
df1.sort_values(by=['VenueCategory'],inplace=True)
df1.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Neighborhood,Company,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
432,East Bay,21Tech,37.803900,-122.270794,Kamdesh Afghan Kabob House,37.80333,-122.268431,Afghan Restaurant
15706,East Bay,Comfy,37.804838,-122.268901,Kamdesh Afghan Kabob House,37.80333,-122.268431,Afghan Restaurant
12386,East Bay,CashBet,37.807846,-122.265079,Kamdesh Afghan Kabob House,37.80333,-122.268431,Afghan Restaurant
17310,East Bay,Daily Kos,37.804364,-122.271114,Kamdesh Afghan Kabob House,37.80333,-122.268431,Afghan Restaurant
6821,East Bay,Babylist,37.807139,-122.272810,Kamdesh Afghan Kabob House,37.80333,-122.268431,Afghan Restaurant


In [17]:
# Creating Dummy Variables for Venue Category
onehot = pd.get_dummies(df1['VenueCategory'],prefix='',prefix_sep='')

In [18]:
# Creating a copy of data frame
df2 = df1

In [19]:
onehot.head()

,Afghan Restaurant,African Restaurant,American Restaurant,Asian Restaurant,Brazilian Restaurant,Burmese Restaurant,Cajun / Creole Restaurant,Cambodian Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Cuban Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,French Restaurant,Greek Restaurant,Hawaiian Restaurant,Hotpot Restaurant,Indian Restaurant,Indonesian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Mongolian Restaurant,New American Restaurant,North Indian Restaurant,Peking Duck Restaurant,Peruvian Restaurant,Ramen Restaurant,Restaurant,Seafood Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
432,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15706,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12386,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
17310,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6821,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [20]:
df2.head()

,Neighborhood,Company,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
432,East Bay,21Tech,37.803900,-122.270794,Kamdesh Afghan Kabob House,37.80333,-122.268431,Afghan Restaurant
15706,East Bay,Comfy,37.804838,-122.268901,Kamdesh Afghan Kabob House,37.80333,-122.268431,Afghan Restaurant
12386,East Bay,CashBet,37.807846,-122.265079,Kamdesh Afghan Kabob House,37.80333,-122.268431,Afghan Restaurant
17310,East Bay,Daily Kos,37.804364,-122.271114,Kamdesh Afghan Kabob House,37.80333,-122.268431,Afghan Restaurant
6821,East Bay,Babylist,37.807139,-122.272810,Kamdesh Afghan Kabob House,37.80333,-122.268431,Afghan Restaurant


In [21]:
# Merging the dummy variables with the original dataframe
df3 = pd.merge(df2,onehot,on=df2.index)

In [22]:
df3.head(50)

,key_0,Neighborhood,Company,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory,Afghan Restaurant,African Restaurant,American Restaurant,Asian Restaurant,Brazilian Restaurant,Burmese Restaurant,Cajun / Creole Restaurant,Cambodian Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Cuban Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,French Restaurant,Greek Restaurant,Hawaiian Restaurant,Hotpot Restaurant,Indian Restaurant,Indonesian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Mongolian Restaurant,New American Restaurant,North Indian Restaurant,Peking Duck Restaurant,Peruvian Restaurant,Ramen Restaurant,Restaurant,Seafood Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,432,East Bay,21Tech,37.803900,-122.270794,Kamdesh Afghan Kabob House,37.803330,-122.268431,Afghan Restaurant,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,15706,East Bay,Comfy,37.804838,-122.268901,Kamdesh Afghan Kabob House,37.803330,-122.268431,Afghan Restaurant,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,12386,East Bay,CashBet,37.807846,-122.265079,Kamdesh Afghan Kabob House,37.803330,-122.268431,Afghan Restaurant,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,17310,East Bay,Daily Kos,37.804364,-122.271114,Kamdesh Afghan Kabob House,37.803330,-122.268431,Afghan Restaurant,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,6821,East Bay,Babylist,37.807139,-122.272810,Kamdesh Afghan Kabob House,37.803330,-122.268431,Afghan Restaurant,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,12625,East Bay,Ceres Imaging,37.803616,-122.271080,Kamdesh Afghan Kabob House,37.803330,-122.268431,Afghan Restaurant,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,6009,East Bay,Ask Media Group,37.803686,-122.275269,Kamdesh Afghan Kabob House,37.803330,-122.268431,Afghan Restaurant,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,8168,East Bay,Binti,37.807875,-122.265068,Kamdesh Afghan Kabob House,37.803330,-122.268431,Afghan Restaurant,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,7241,East Bay,Beezwax Datatools,37.805766,-122.272202,Kamdesh Afghan Kabob House,37.803330,-122.268431,Afghan Restaurant,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,18121,East Bay,Design Action Collective,37.806559,-122.267826,Kamdesh Afghan Kabob House,37.803330,-122.268431,Afghan Restaurant,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
df3.drop(['key_0'],axis=1,inplace=True)
df3.head()

,Neighborhood,Company,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory,Afghan Restaurant,African Restaurant,American Restaurant,Asian Restaurant,Brazilian Restaurant,Burmese Restaurant,Cajun / Creole Restaurant,Cambodian Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Cuban Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,French Restaurant,Greek Restaurant,Hawaiian Restaurant,Hotpot Restaurant,Indian Restaurant,Indonesian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Mongolian Restaurant,New American Restaurant,North Indian Restaurant,Peking Duck Restaurant,Peruvian Restaurant,Ramen Restaurant,Restaurant,Seafood Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,East Bay,21Tech,37.803900,-122.270794,Kamdesh Afghan Kabob House,37.80333,-122.268431,Afghan Restaurant,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,East Bay,Comfy,37.804838,-122.268901,Kamdesh Afghan Kabob House,37.80333,-122.268431,Afghan Restaurant,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,East Bay,CashBet,37.807846,-122.265079,Kamdesh Afghan Kabob House,37.80333,-122.268431,Afghan Restaurant,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,East Bay,Daily Kos,37.804364,-122.271114,Kamdesh Afghan Kabob House,37.80333,-122.268431,Afghan Restaurant,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,East Bay,Babylist,37.807139,-122.272810,Kamdesh Afghan Kabob House,37.80333,-122.268431,Afghan Restaurant,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
group_cluster = df3.drop(['Neighborhood','Company','Latitude','Longitude','VenueName','VenueLatitude','VenueLongitude','VenueCategory'],axis=1)
group_cluster.head()

,Afghan Restaurant,African Restaurant,American Restaurant,Asian Restaurant,Brazilian Restaurant,Burmese Restaurant,Cajun / Creole Restaurant,Cambodian Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Cuban Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,French Restaurant,Greek Restaurant,Hawaiian Restaurant,Hotpot Restaurant,Indian Restaurant,Indonesian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Mongolian Restaurant,New American Restaurant,North Indian Restaurant,Peking Duck Restaurant,Peruvian Restaurant,Ramen Restaurant,Restaurant,Seafood Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Using k-means algorithm for the clustering of the restaurants near the offices.

In [26]:
k=5
kmeans = KMeans(n_clusters=k,random_state=0).fit(group_cluster)
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [27]:
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [28]:
df3.insert(0,'ClusterLabels',kmeans.labels_)

In [29]:
df3.head()

,ClusterLabels,Neighborhood,Company,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory,Afghan Restaurant,African Restaurant,American Restaurant,Asian Restaurant,Brazilian Restaurant,Burmese Restaurant,Cajun / Creole Restaurant,Cambodian Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Cuban Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,French Restaurant,Greek Restaurant,Hawaiian Restaurant,Hotpot Restaurant,Indian Restaurant,Indonesian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Mongolian Restaurant,New American Restaurant,North Indian Restaurant,Peking Duck Restaurant,Peruvian Restaurant,Ramen Restaurant,Restaurant,Seafood Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,0,East Bay,21Tech,37.803900,-122.270794,Kamdesh Afghan Kabob House,37.80333,-122.268431,Afghan Restaurant,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,East Bay,Comfy,37.804838,-122.268901,Kamdesh Afghan Kabob House,37.80333,-122.268431,Afghan Restaurant,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,East Bay,CashBet,37.807846,-122.265079,Kamdesh Afghan Kabob House,37.80333,-122.268431,Afghan Restaurant,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,East Bay,Daily Kos,37.804364,-122.271114,Kamdesh Afghan Kabob House,37.80333,-122.268431,Afghan Restaurant,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,East Bay,Babylist,37.807139,-122.272810,Kamdesh Afghan Kabob House,37.80333,-122.268431,Afghan Restaurant,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Creating the map of all the restaurants along with their cuisine specialities and plotting all the companies on the same map for evaluation purpose. The clusters in the map shows different types of restaurants on the basis of different attributes, mainly cuisine.

In [31]:
# create map

map_clusters = folium.Map(location=[37.7749, -122.4194], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, name, cat, cluster in zip(df3['VenueLatitude'], df3['VenueLongitude'], df3['VenueName'],df3['VenueCategory'], df3['ClusterLabels']):
    label = folium.Popup(name +','+ cat + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=4,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
for lat, lng, company, neighbourhood in zip(df3['Latitude'], df3['Longitude'], df3['Company'], df3['Neighborhood']):
    label = '{}, {}'.format(company, neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters) 

    

    
map_clusters

Examining Clusters

In [32]:
cluster0 = df3.loc[df3['ClusterLabels'] == 0, df3.columns[[1] + list(range(5, df3.shape[1]))]]


The Cluster 0 is shown in Red Colour and contains the most number of restaurants, mainly Japanese, Sushi, Indian, etc.

In [33]:
cluster0['VenueCategory'].value_counts()

Japanese Restaurant                50
Sushi Restaurant                   42
Indian Restaurant                  36
Fast Food Restaurant               26
Vegetarian / Vegan Restaurant      25
Mediterranean Restaurant           23
Thai Restaurant                    23
Restaurant                         20
Caribbean Restaurant               19
Brazilian Restaurant               19
Tapas Restaurant                   18
Seafood Restaurant                 18
Asian Restaurant                   18
Italian Restaurant                 17
New American Restaurant            15
Ramen Restaurant                   14
Hotpot Restaurant                  14
Afghan Restaurant                  12
Cambodian Restaurant                9
Korean Restaurant                   9
Burmese Restaurant                  8
Dim Sum Restaurant                  8
Taiwanese Restaurant                8
French Restaurant                   8
Dumpling Restaurant                 7
Greek Restaurant                    7
Mongolian Re

The Cluster 1 is shown in Purple Colour and contains 85 restaurants of Mexican cuisine.

In [34]:
cluster1 = df3.loc[df3['ClusterLabels'] == 1, df3.columns[[1] + list(range(5, df3.shape[1]))]]
cluster1['VenueCategory'].value_counts()

Mexican Restaurant    85
Name: VenueCategory, dtype: int64

The Cluster 2 is shown in Light Blue Color and contains 58 American Restaurants. Cluster 3 is shown in Green color and contains 69 Chinese Restaurants. Cluster 4 is shown in Orange color and contains 65 Vietnamese Restaurants.

In [37]:
cluster3 = df3.loc[df3['ClusterLabels'] == 2, df3.columns[[1] + list(range(5, df3.shape[1]))]]
cluster4 = df3.loc[df3['ClusterLabels'] == 3, df3.columns[[1] + list(range(5, df3.shape[1]))]]
cluster5 = df3.loc[df3['ClusterLabels'] == 4, df3.columns[[1] + list(range(5, df3.shape[1]))]]

In [38]:
cluster3['VenueCategory'].value_counts()

American Restaurant    58
Name: VenueCategory, dtype: int64

In [39]:
cluster4['VenueCategory'].value_counts()

Chinese Restaurant    69
Name: VenueCategory, dtype: int64

In [40]:
cluster5['VenueCategory'].value_counts()

Vietnamese Restaurant    65
Name: VenueCategory, dtype: int64

The observations that can be made from the map are:

North Bay has very few restaurants. But interestingly these few restaurants cover various cuisines. Simply judging by the number of restaurants, just opening one another restaurant regardless of cuisines might not be a bad idea. 

In East Bay area near Art.com, there is only one restaurant nearby. Several other restaurants in Cluster 0 are a bit far away. So opening a new restaurant as long as a Mexican restaurant shall be a good idea. 

In Chinatown near Okaland City Center, there are quite many Chinese restaurants and few Far East restaurants. This is a good at least. As other cuisines might not be successful in Chinatown area. 

Around Oakland City Center, there are no Mexican restaurants. 

In the San Lendro region, there is no American restaurant, so opening one there might lead to a good business. 

In San Ramon Marketplace Shopping Center, there is no Mexican restaurant in their food court.

Many restaurants are located in South Bay area, thus a lot more observations can be drawn here. 